In [2]:
from glob import glob
from sklearn.model_selection import GroupKFold
import cv2
import os
import time
import random
import pandas as pd
import numpy as np


# from mlxtend.plotting import plot_learning_curves
# plot_learning_curves

!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets, models, transforms
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
# from PIL import Image

from sklearn import metrics
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import learning_curve
import seaborn as sn
import matplotlib.pyplot as plt

BASE_PATH = "/kaggle/input/alaska2-image-steganalysis"
DATA_ROOT_PATH = '../input/alaska2-image-steganalysis'
RESNET_MODEL_PATH = '../input/newresnetmodel/resnet18modelv1.pth'

In [ ]:
#Notes - to load resnet18
#resnet = load_model('resnet18')
# resnet = load_checkpoint(resnet, RESNET_MODEL_PATH).to(device)

# Utilities

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def load_model(name):
    """
    Load basic resnet34/ resnet18 from the internet
    """
    #loading resnet34
    if name == 'resnet34':
        model = models.resnet34(pretrained=True)
    elif name == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif name == 'effnet':
        model = EfficientNet.from_pretrained('efficientnet-b0')
        
    #to only finteune the top layer of the model
    for param in model.parameters():
        param.requires_grad = False


    # Replace the top layer for finetuning.
    model.fc = nn.Linear(model.fc.in_features, 4)  # 100 is an example.
    model = model.to(device)
    return model

In [4]:
effnet = load_model('effnet')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


AttributeError: 'EfficientNet' object has no attribute 'fc'

In [ ]:
def load_checkpoint(model, filepath):
    """
    Loads the weights into the model
    """
    states_weights = torch.load(filepath, map_location={'cuda:0': 'cpu'})
    model.load_state_dict(states_weights)
    #model = checkpoint['model']
#     model.load_state_dict(checkpoint['state_dict'])
#     for parameter in model.parameters():
#         parameter.requires_grad = False
    
#     model.eval()
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    return model

In [ ]:
def show_image(idx):
    image, target = train_dataset[idx]
    numpy_image = image.permute(1,2,0).cpu().numpy()

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    ax.set_axis_off()
    ax.imshow(numpy_image)

#show_image(0)

# Train and Cross Validation

In [ ]:
dataset = []

for label, kind in enumerate(['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']):
    for path in glob('../input/alaska2-image-steganalysis/Cover/*.jpg'):
        dataset.append({
            'kind': kind,
            'image_name': path.split('/')[-1],
            'label': label
        })

random.shuffle(dataset)
dataset = pd.DataFrame(dataset)
gkf = GroupKFold(n_splits=5)
dataset.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(gkf.split(X=dataset.index, y=dataset['label'], groups=dataset['image_name'])):
    dataset.loc[dataset.iloc[val_index].index, 'fold'] = fold_number



In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, kinds, image_names, labels, transforms=None):
        super().__init__()
        self.kinds = kinds
        self.image_names = image_names
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index: int):
        kind, image_name, label = self.kinds[index], self.image_names[index], self.labels[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{kind}/{image_name}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
            
        target = label
        return image, target

    def __len__(self) -> int:
        return self.image_names.shape[0]

    def get_labels(self):
        return list(self.labels)

In [ ]:
def get_train_transforms():
    return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
fold_number = 0
train_dataset = DatasetRetriever(
    kinds=dataset[dataset['fold'] != fold_number].kind.values,
    image_names=dataset[dataset['fold'] != fold_number].image_name.values,
    labels=dataset[dataset['fold'] != fold_number].label.values,
    transforms=get_train_transforms(),
)

validation_dataset = DatasetRetriever(
    kinds=dataset[dataset['fold'] == fold_number].kind.values,
    image_names=dataset[dataset['fold'] == fold_number].image_name.values,
    labels=dataset[dataset['fold'] == fold_number].label.values,
    transforms=get_valid_transforms(),
)

In [ ]:
num_epochs = 5
batch_size = 16
learning_rate = 0.01

In [ ]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

class RunBuilder():
    @staticmethod
    def get_runs(params):
        
        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        
        return runs

In [ ]:
class Epoch():
    def __init__(self):
        self.count = 0
        self.loss = 0
        self.start_time = None
    
class Run():
    def __init__(self):
        self.params = None
        self.count = 0
        self.data = []
        self.start_time = None

In [ ]:
class LabelsAndPreds():
    @staticmethod
    def transform_labels_for_metric(labels):
        
        return torch.clamp(labels, min=0, max=1).detach().numpy()
        
        
    @staticmethod
    def transform_preds_for_metric(preds):
        
        preds = nn.Softmax()(preds)
        mx, indices = torch.max(preds, 1)
        mask = (indices!=0).to(torch.float) - (indices==0).to(torch.float) 
        preds = mask*mx
        
        return preds.detach().numpy()

In [ ]:
from IPython.display import clear_output, display

class RunManager():
    def __init__(self):
        self.epoch = Epoch()
        self.run = Run()
        
        self.preds = None
        self.labels = None
        self.network = None
        self.loader = None
        self.runs_losses = []
    
    
    def begin_run(self, run, network, loader):
        
        self.run.start_time = time.time()
        self.run.params = run
        self.run.count += 1
        
        self.network = network
        self.loader = loader
        
        self.preds = torch.empty(0)
        self.labels = torch.empty(0)        
        
        
    def end_run(self):
        self.epoch.count = 0
        
        
    def begin_epoch(self):
        self.epoch.start_time = time.time()
        self.epoch.count += 1
        self.epoch.loss = 0
        
    
    def end_epoch(self):
        epoch_duration = time.time() - self.epoch.start_time
        run_duration = time.time() - self.run.start_time
        
        loss = self.epoch.loss / len(self.loader.dataset)
        
        score = self.alaska_auc()
        results = OrderedDict()
        results ["run_count"] = self.run.count
        results['epoch_count'] = self.epoch.count
        results['loss'] = loss
        results['score'] = score
        results['epoch_duration'] = epoch_duration
        results['run_duration'] = run_duration
        
        for k,v in self.run.params._asdict().items():
            results[k] = v #eg: results['lr'] = 0.1
        
        self.run.data.append(results)
        df = pd.DataFrame.from_dict(self.run.data, orient='columns')
        
        clear_output(wait=True)
        display(df)
    
    
    def track_loss(self, loss):
        self.epoch.loss += loss.item() * self.loader.batch_size
        
        
    def track_metric(self, labels, preds):
        self.labels = torch.cat((self.labels, labels.cpu().to(torch.float32)))
        self.preds = torch.cat((self.preds, preds.cpu()))
        
        
    def save_and_return_df(self, fileName): #doesn't work
        
        df = pd.DataFrame.from_dict(
            self.run.data
            ,orient='columns'
        )
        df.to_csv(f'{fileName}.csv')
#         with open(f'{fileName}.json','w',encoding='utf-8') as f:
#             json.dump(self.run.data, f, ensure_ascii=False, indent=4)
        
        return df    
        
    def alaska_auc(self):
        """"
        return the alaska_metric error
        """
        if len(self.labels) == 0 or len(self.preds) == 0:
            return -1
        labels = LabelsAndPreds.transform_labels_for_metric(self.labels)
        preds = LabelsAndPreds.transform_preds_for_metric(self.preds)
        
        tpr_thresholds = [0.0, 0.4, 1.0]
        weights = [2, 1]
        fpr,tpr,thresholds= metrics.roc_curve(labels,preds)

        areas=np.subtract(tpr_thresholds[1:],tpr_thresholds[:2])
        normalization=np.dot(areas,weights)
        competition_metric = 0
        for idx, weight in enumerate(weights):
            y_min = tpr_thresholds[idx]
            y_max = tpr_thresholds[idx + 1]
            mask = (y_min < tpr) & (tpr < y_max)

            if(mask.sum()==0):
                continue
            x_padding = np.linspace(fpr[mask][-1], 1, 100)
            x = np.concatenate([fpr[mask], x_padding])
            y = np.concatenate([tpr[mask], [y_max] * len(x_padding)])
            y = y - y_min # normalize such that curve starts at y=0
            score = metrics.auc(x, y)
            submetric = score * weight
            best_subscore = (y_max - y_min) * weight
            competition_metric += submetric
        return competition_metric / normalization
    
    def plot_confusion_matrix(self):
        """
        Plots the confusion matrix between labels and predictions.
        Labels are multiclass. Eg: [0,1,2,2,3,1,2,1,0]
        Predictions are multiclass. Eg: [0,1,2,2,3,1,2,1,0]
        len(labels) == len(predictions)

        @param labels - the truths. Numpy array
        @param predictions - the predictions by your model. Numpy array
        """
        assert len(self.labels) == len(self.preds)
        names = ('Cover', 'JMiPOD', 'JUNIWARD', 'UERD')
        mutliclass_preds = torch.argmax(self.preds,dim = 1)
        cm = confusion_matrix(self.labels.detach().numpy(), mutliclass_preds.detach().numpy())
        df_cm = pd.DataFrame(cm, index = [i for i in names],
                          columns = [i for i in names])
        plt.figure(figsize = (10,7))
        drawing = sn.heatmap(df_cm, annot=True)
        drawing.set(xlabel='Predictions', ylabel='True')
    
    def get_loss(self):
        return self.epoch.loss / len(self.loader.dataset) #the loss for the last epoch is the loss for the run

In [ ]:
def train(manager, params, num_epochs):
    
    for run in RunBuilder.get_runs(params):
        #Loss and optimizer
        criterion = nn.CrossEntropyLoss().to(device)
        optimizer = torch.optim.Adam(run.network.parameters(), lr=run.lr)
        
        altered_train_dataset = torch.utils.data.Subset(train_dataset, list(range(run.m)))
        train_loader = torch.utils.data.DataLoader(dataset=altered_train_dataset
                                                   ,batch_size=run.batch_size 
                                                   ,shuffle=True
                                                   ,num_workers = 2)
        
        manager.begin_run(run, run.network, train_loader)
        for epoch in range(num_epochs):
            manager.begin_epoch()
            for i, batch in enumerate(train_loader):
                images = batch[0].to(device)
                labels = batch[1].to(device)
                preds = run.network(images)
                loss = criterion(preds, labels)
                optimizer.zero_grad()
                loss.backward()            
                optimizer.step()

                manager.track_loss(loss)
#                 manager.track_metric(labels, preds)
            
            manager.end_epoch()
            
        manager.end_run()
    
    print('Finished Training')
    df = manager.save_and_return_df('train_results')
    run_loss = manager.get_loss()
    return df, run_loss

    

In [ ]:
def cross_validate(manager, params):
    """
    """ 
    with torch.no_grad():
        for run in RunBuilder.get_runs(params):
            
            criterion = nn.CrossEntropyLoss().to(device)
            
            altered_validation_dataset = torch.utils.data.Subset(validation_dataset, list(range(run.m)))

            validation_loader = torch.utils.data.DataLoader(dataset=altered_validation_dataset,
                                           batch_size=run.batch_size, 
                                           shuffle=True,
                                           num_workers = 2)
            
            manager.begin_run(run, run.network, validation_loader)
            manager.begin_epoch() #only one epoch needed in cross validation set
            for i, batch in enumerate(validation_loader):
                images = batch[0].to(device)
                labels = batch[1].to(device)
                preds = run.network(images)
                loss = criterion(preds, labels)

                manager.track_loss(loss)
#                 manager.track_metric(labels, preds)

            manager.end_epoch()
            manager.end_run()

    print('Finished Validating')
    df = manager.save_and_return_df('train_results')
    run_loss = manager.get_loss()

    return df, run_loss


In [ ]:
resnet = load_model('effnet')

In [ ]:
train_params = OrderedDict(
        network = [resnet]
        ,m = [len(train_dataset)] # len(train_dataset) for full training
        ,lr = [.001]
        ,batch_size = [8]
)
train_manager = RunManager()
train_df, train_loss = train(train_manager, train_params, num_epochs=7)
train_df.sort_values(by='score', ascending=False)

In [ ]:
cv_params = OrderedDict(
        network = [resnet]
        ,m = [len(validation_dataset)] #len(validation_dataset) for full validation
        ,batch_size = [8]
)
cv_manager = RunManager()
cv_df, _ = cross_validate(cv_manager, cv_params)
cv_df.sort_values(by='score', ascending=False)
cv_manager.plot_confusion_matrix()

# Diagnosing model

In [ ]:
def diagnose(set_sizes):
    
    train_params = OrderedDict(
        network = None
        ,m = None #the first m examples of dataset to train on.
        ,lr = [.001]
        ,batch_size = [16]
        )
    cv_params = OrderedDict(
        network = None
        ,m = None #the first m examples of dataset to train on
        ,batch_size = [32]
    )
    train_losses = []
    cv_losses = []
    models = []
    for m in set_sizes:
        resnet = load_model('resnet34')
        
        train_params['network'] = [resnet]
        train_params['m'] = [m]
        cv_params['network'] = [resnet]
        cv_params['m'] = [m]
        
        train_manager = RunManager()
        cv_manager = RunManager()
        
        _, train_loss = train(train_manager, train_params, num_epochs=7)
        _, cv_loss = cross_validate(cv_manager, cv_params)
        train_losses.append(train_loss)
        cv_losses.append(cv_loss)
        models.append(resnet)
    return models, train_losses, cv_losses
        

In [ ]:
def draw_learning_curves(set_sizes, train_losses, cv_losses):
    """
    @param set_sizes: list containing the different set sizes used
    @param train_losses: the train loss corresponding to the set sizes
    @param cv_losses: the cv loss corresponding to the set sizes
    
    When training and validating on train and validation sets with size set_sizes[i],
    the train_loss is  train_losses[i] and cv loss is cv_loss[i]
    """
    # plt.plot(train_set_sizes, train_runs_losses)
    plt.plot(set_sizes, train_losses, label='Train Loss')
    plt.plot(set_sizes, cv_losses, label='CV Loss')
    plt.legend()
    plt.show()

In [ ]:
# set_sizes = [i for i in range(0, len(validation_dataset)+1, 10000)]
# set_sizes[0] = 1
# models, train_losses, cv_losses  = diagnose(set_sizes)

In [ ]:
# print(train_losses)
# print(cv_losses)

In [ ]:
# set_sizes = [10000, 20000, 30000, 40000, 50000, 60000] 
# train_losses = [1.3995465944290162, 1.4217510432243348, 1.4290748152414958, 1.4270348804950714, 1.428084383506775, 1.4317004181861877] 
# cv_losses = [1.4482383392333984, 1.4100874675750732, 1.4082810043334961, 1.4243976655960082, 1.4023886753082275, 1.420537694867452]
# draw_learning_curves(set_sizes, train_losses, cv_losses)

# print(len(models)) [ model1, model2,...]

In [ ]:
# draw_learning_curves(set_sizes, train_losses, cv_losses)

# Testing model

In [ ]:
class DatasetSubmissionRetriever(Dataset):

    def __init__(self, image_names, transforms=None):
        super().__init__()
        self.image_names = image_names
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_name = self.image_names[index]
        image = cv2.imread(f'{BASE_PATH}/Test/{image_name}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        return image_name, image

    def __len__(self) -> int:
        return self.image_names.shape[0]

In [ ]:
test_dataset = DatasetSubmissionRetriever(
    image_names=np.array([path.split('/')[-1] for path in glob('../input/alaska2-image-steganalysis/Test/*.jpg')]),
    transforms=get_valid_transforms(),
)

In [ ]:
def test(network):
    """
    Return array containing the predictions
    """
    test_data_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=2,
        drop_last=False,
    )
    total_step = len(test_data_loader)
    result = {'Id': [], 'Label': []}
    with torch.no_grad():
        for i, (image_names, images) in enumerate(test_data_loader):
            print(f'Progress: {round(100*i/(total_step-1),1)}%', end='\r')
    
            preds = network(images.to(device))
            preds = preds.cpu()
            preds = LabelsAndPreds.transform_preds_for_metric(preds)
            
            result['Id'].extend(image_names)
            result['Label'].extend(preds)

    return result

In [ ]:
result = test(resnet)

In [ ]:
submission = pd.DataFrame(result)
submission.to_csv('submission.csv', index=False)
submission.head()